In [ ]:
# Install easyocr
!pip install easyocr

In [7]:
import easyocr
import cv2
import matplotlib.pyplot as plt
import time

## Using GPU

In [ ]:
# Initialize easyocr reader with GPU
reader = easyocr.Reader(['en'], gpu=True)

# Read the video file
video_path = '/content/sample_data/WhatsApp Video 2024-10-11 at 17.20.27_3339e0f0.mp4'
cap = cv2.VideoCapture(video_path)

correct_text_predictions = 0
total_text_predictions = 0

# Process every frame
frame_rate = 1  # Process every frame on GPU
frame_count = 0

# Measure time
start_time = time.time()

while cap.isOpened():
    ret, frame = cap.read()

    if not ret:
        break

    # Process every frame (no frame skipping)
    if frame_count % frame_rate == 0:
        text_ = reader.readtext(frame)

        for bbox, text, score in text_:
          total_text_predictions += 1
          if score > 0.25:
            correct_text_predictions += 1
            top_left = (int(bbox[0][0]), int(bbox[0][1]))
            bottom_right = (int(bbox[2][0]), int(bbox[2][1]))
            cv2.rectangle(frame, top_left, bottom_right, (0, 255, 0), 2)

            org = (int(bbox[0][0]), int(bbox[0][1]) - 5)
            cv2.putText(frame, text, org, cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 0, 255), 2)

        plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        plt.show()

    frame_count += 1

end_time = time.time()

elapsed_time = end_time - start_time
print(f"Elapsed time: {elapsed_time} seconds")

gpu_fps = frame_count / elapsed_time
print(f"Frame rate: {gpu_fps} frames per second")

gpu_accuracy = correct_text_predictions / total_text_predictions
print(f"Accuracy: {gpu_accuracy}")


# Using CPU
## Technique: Skipping the frames

In [ ]:
# Using cpu

# The technique: Skipping the frames


# Initialize easyocr reader with GPU
reader = easyocr.Reader(['en'], gpu=False)

# Read the video file
video_path = '/content/sample_data/WhatsApp Video 2024-10-11 at 17.20.27_3339e0f0.mp4'
cap = cv2.VideoCapture(video_path)


correct_text_predictions = 0
total_text_predictions = 0

# Process every frame
frame_rate = 10  # Can be adjusted
frame_count = 0

# Measure time
start_time = time.time()

while cap.isOpened():
    ret, frame = cap.read()

    if not ret:
        break

    # Process every 10th frame
    if frame_count % frame_rate == 0:
        text_ = reader.readtext(frame)

        for bbox, text, score in text_:
          total_text_predictions += 1
          if score > 0.25:  # Filter predictions with score threshold
            correct_text_predictions += 1
            top_left = (int(bbox[0][0]), int(bbox[0][1]))
            bottom_right = (int(bbox[2][0]), int(bbox[2][1]))
            cv2.rectangle(frame, top_left, bottom_right, (0, 255, 0), 2)

            org = (int(bbox[0][0]), int(bbox[0][1]) - 5)
            cv2.putText(frame, text, org, cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 0, 255), 2)

        plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        plt.show()

    frame_count += 1

end_time = time.time()

elapsed_time = end_time - start_time
print(f"Elapsed time: {elapsed_time} seconds")

cpu_fps = frame_count / elapsed_time
print(f"Frame rate: {cpu_fps} frames per second")

cpu_accuracy = correct_text_predictions / total_text_predictions
print(f"Accuracy: {cpu_accuracy}")


# Visualization of FPS and accuracy metrics

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

labels = ['GPU', 'CPU (Frame Skipping)']

# FPS Visualization
plt.figure(figsize=(10, 6))
plt.bar(labels, [gpu_fps, cpu_fps], color='tab:blue', alpha=0.7)
plt.xlabel('Model')
plt.ylabel('FPS')
plt.title('FPS Comparison: GPU vs CPU (Frame Skipping)')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

# Accuracy Visualization
plt.figure(figsize=(10, 6))
plt.plot(labels, [gpu_accuracy, cpu_accuracy], color='tab:green', marker='o', linewidth=2, label='Accuracy')
plt.xlabel('Model')
plt.ylabel('Accuracy')
plt.title('Accuracy Comparison: GPU vs CPU (Frame Skipping)')
plt.grid(True)
plt.ylim([0, 1])  
plt.tight_layout()
plt.legend()
plt.show()
